In [6]:
import psycopg2

# Connect to PostgreSQL database
def connect():
    return psycopg2.connect(
        host="localhost",
        database="hospital_har", 
        user="postgres",    
        password="harshith"
    )

# Create tables
def create_tables():
    commands = (
        """
        CREATE TABLE IF NOT EXISTS Patient (
            patient_id SERIAL PRIMARY KEY,
            first_name VARCHAR(100) NOT NULL,
            last_name VARCHAR(100) NOT NULL,
            gender VARCHAR(10),
            age INT,
            phone_number VARCHAR(15),
            email VARCHAR(100) UNIQUE,
            address TEXT,
            password VARCHAR(255) NOT NULL,
            medical_history TEXT
        );
        """,
        """
        CREATE TABLE IF NOT EXISTS Doctor (
            doctor_id SERIAL PRIMARY KEY,
            first_name VARCHAR(100) NOT NULL,
            last_name VARCHAR(100) NOT NULL,
            specialization VARCHAR(100) NOT NULL,
            phone_number VARCHAR(15),
            email VARCHAR(100) UNIQUE,
            address TEXT,
            password VARCHAR(255) NOT NULL,
            specialty VARCHAR(100),        
            experience INT,                
            appointment_price DECIMAL(10, 2)  
        );
        """,
        """
        CREATE TABLE IF NOT EXISTS Manager (
            manager_id SERIAL PRIMARY KEY,
            first_name VARCHAR(100) NOT NULL,
            last_name VARCHAR(100) NOT NULL,
            phone_number VARCHAR(15),
            email VARCHAR(100) UNIQUE,
            password VARCHAR(255) NOT NULL
        );
        """,
        """
        CREATE TABLE IF NOT EXISTS Appointment (
            appointment_id SERIAL PRIMARY KEY,
            patient_id INT REFERENCES Patient(patient_id),
            doctor_id INT REFERENCES Doctor(doctor_id),
            appointment_date DATE NOT NULL,
            appointment_time TIME NOT NULL,
            status VARCHAR(50),
            medical_report TEXT , 
            reason TEXT
        );
        """,
        """
        CREATE TABLE IF NOT EXISTS Medical_Records (
            record_id SERIAL PRIMARY KEY,
            appointment_id INT REFERENCES Appointment(appointment_id),
            doctor_review TEXT,           
            diagnosis VARCHAR(255),       
            prescription TEXT,           
            follow_up_date DATE           
        );
        """,
        """
        CREATE TABLE IF NOT EXISTS Staff (
            staff_id SERIAL PRIMARY KEY,
            name VARCHAR(100),
            role VARCHAR(50),             
            contact_info VARCHAR(100)
        );
        """
        """
        CREATE TABLE IF NOT EXISTS Receptionist (
            receptionist_id SERIAL PRIMARY KEY,
            first_name VARCHAR(100) NOT NULL,
            last_name VARCHAR(100) NOT NULL,
            phone_number VARCHAR(15),
            email VARCHAR(100) UNIQUE,
            password VARCHAR(255) NOT NULL,
            shift_time VARCHAR(50)
        );
        """
        """
        CREATE TABLE IF NOT EXISTS Bills (
            bill_id SERIAL PRIMARY KEY,
            appointment_id INT REFERENCES Appointment(appointment_id) ON DELETE CASCADE,
            total_amount DECIMAL(10, 2) NOT NULL,
            payment_status VARCHAR(50) DEFAULT 'Unpaid',
            payment_date DATE,
            payment_method VARCHAR(50),
            receptionist_id INT REFERENCES Receptionist(receptionist_id) ON DELETE SET NULL  
        );
        """
    )
    
    conn = None
    try:
        conn = connect()
        cur = conn.cursor()
        # Execute each SQL command to create tables
        for command in commands:
            cur.execute(command)
        conn.commit()  # Save the changes
        cur.close()
        print("Tables created successfully.")
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()

if __name__ == '__main__':
    create_tables()


Tables created successfully.


In [2]:
import psycopg2
import getpass
import re
def connect():
    return psycopg2.connect(
        host="localhost",
        database="hospital_har",  
        user="postgres",      
        password="harshith"   
    )

def authenticate_user(cursor, role, email, password):
    if role == 'manager':
        cursor.execute("SELECT * FROM Manager WHERE email = %s AND password = %s;", (email, password))
    elif role == 'doctor':
        cursor.execute("SELECT * FROM Doctor WHERE email = %s AND password = %s;", (email, password))
    elif role == 'patient':
        cursor.execute("SELECT * FROM Patient WHERE email = %s AND password = %s;", (email, password))
    user = cursor.fetchone()
    
    if user:
        return user
    else:
        return None

def validate_email(email):
    pattern = r'^[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+$'
    if re.match(pattern, email):
        return True
    else:
        return False

def register_patient(cursor):
    first_name = input("Enter your first name: ")
    last_name = input("Enter your last name: ")
    gender = input("Enter your gender: ")
    age = int(input("Enter your age: "))
    phone_number = input("Enter your phone number: ")

    # Email validation
    while True:
        email = input("Enter your email (format: example@gmail.com): ")
        if validate_email(email):
            break
        else:
            print("Invalid email format. Please try again.")

    # Password input with dots
    password = getpass.getpass("Enter your password: ")
    address = input("Enter your address: ")
    medical_history = input("Enter your medical history: ")

    cursor.execute("""
        INSERT INTO Patient (first_name, last_name, gender, age, phone_number, email, password, address, medical_history)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s);
    """, (first_name, last_name, gender, age, phone_number, email, password, address, medical_history))

    print("Registration successful!")

def is_time_valid(cursor, doctor_id, appointment_date, appointment_time):
    
    if not ('09:00' <= appointment_time <= '16:30'):
        print("Appointment time must be between 9:00 AM and 4:30 PM.")
        return False
    
    cursor.execute("""
        SELECT * FROM Appointment 
        WHERE doctor_id = %s AND appointment_date = %s 
        AND (appointment_time BETWEEN %s AND %s);
    """, (doctor_id, appointment_date, appointment_time, f'{int(appointment_time[:2])+1}:{appointment_time[3:]}'))

    if cursor.fetchone():
        print(f"Doctor already has an appointment around {appointment_time}. Please choose a different time.")
        return False

    return True

def book_appointment(cursor, patient_id):
    specialization = input("Enter the specialization you want: ")
    reason = input("Enter the reason for the appointment: ")

    cursor.execute("""
        SELECT doctor_id, first_name, last_name, experience, appointment_price
        FROM Doctor
        WHERE specialization ILIKE %s;
    """, (specialization,))

    doctors = cursor.fetchall()

    if not doctors:
        print("No doctors found with that specialization.")
        return

    print("Available Doctors:")
    for index, doctor in enumerate(doctors):
        print(f"{index + 1}. Dr. {doctor[1]} {doctor[2]}, Experience: {doctor[3]} years, Price: {doctor[4]}")

    doctor_choice = int(input("Select a doctor by number: ")) - 1

    if doctor_choice < 0 or doctor_choice >= len(doctors):
        print("Invalid choice.")
        return

    selected_doctor = doctors[doctor_choice]
    doctor_id = selected_doctor[0]

    appointment_date = input("Enter appointment date (YYYY-MM-DD): ")

    while True:
        appointment_time = input("Enter appointment time (HH:MM) between 09:00 AM and 16:30 PM: ")
        if is_time_valid(cursor, doctor_id, appointment_date, appointment_time):
            break

    cursor.execute("""
        INSERT INTO Appointment (patient_id, doctor_id, appointment_date, appointment_time, status, reason)
        VALUES (%s, %s, %s, %s, %s, %s);
    """, (patient_id, doctor_id, appointment_date, appointment_time, 'Scheduled', reason))

    print("Appointment booked successfully.")



def view_appointments(cursor, patient_id):
    cursor.execute("""
    SELECT a.appointment_id, d.first_name, d.last_name, a.appointment_date, a.appointment_time, a.status, a.medical_report
    FROM Appointment a
    JOIN Doctor d ON a.doctor_id = d.doctor_id
    WHERE a.patient_id = %s;
    """, (patient_id,))

    appointments = cursor.fetchall()

    if not appointments:
        print("No appointments found.")
        return

    print("Your Appointments:")
    for appt in appointments:
        print(f"ID: {appt[0]}, Doctor: Dr. {appt[1]} {appt[2]}, Date: {appt[3]}, Time: {appt[4]}, Status: {appt[5]}, Medical_Report: {appt[6]}")

def change_appointment(cursor, patient_id):
    appointment_id = input("Enter the appointment ID you want to change: ")
    new_date = input("Enter new appointment date (YYYY-MM-DD): ")
    new_time = input("Enter new appointment time (HH:MM): ")

    cursor.execute("""
        UPDATE Appointment
        SET appointment_date = %s, appointment_time = %s
        WHERE appointment_id = %s AND patient_id = %s;
    """, (new_date, new_time, appointment_id, patient_id))

    if cursor.rowcount > 0:
        print("Appointment changed successfully.")
    else:
        print("Appointment ID not found or does not belong to you.")

# Function to cancel an appointment
def cancel_appointment(cursor, patient_id):
    appointment_id = input("Enter the appointment ID you want to cancel: ")

    cursor.execute("""
        DELETE FROM Appointment
        WHERE appointment_id = %s AND patient_id = %s;
    """, (appointment_id, patient_id))

    if cursor.rowcount > 0:
        print("Appointment canceled successfully.")
    else:
        print("Appointment ID not found or does not belong to you.")
        
def check_medical_record(cursor, patient_id):
    cursor.execute("""
        SELECT a.appointment_id, d.first_name, d.last_name, a.appointment_date, a.status
        FROM Appointment a
        JOIN Doctor d ON a.doctor_id = d.doctor_id
        WHERE a.patient_id = %s AND a.status = 'Completed'
        ORDER BY a.appointment_date DESC
    """, (patient_id,))
    
    appointments = cursor.fetchall()
    
    if appointments:
        print("Your past appointments:")
        for appointment in appointments:
            print(f"Appointment ID: {appointment[0]}, Doctor: Dr. {appointment[1]} {appointment[2]}, Date: {appointment[3]}, Status: {appointment[4]}")
        
        appointment_id = input("Enter the Appointment ID to view the medical record: ").strip()

        cursor.execute("""
            SELECT mr.diagnosis, mr.doctor_review, mr.prescription, mr.follow_up_date
            FROM Medical_Records mr
            JOIN Appointment a ON mr.appointment_id = a.appointment_id
            WHERE a.appointment_id = %s AND a.patient_id = %s
        """, (appointment_id, patient_id))
        
        medical_record = cursor.fetchone()
        
        if medical_record:
            print(f"\nMedical Record for Appointment ID {appointment_id}:")
            print(f"Diagnosis: {medical_record[0]}")
            print(f"Doctor's Review: {medical_record[1]}")
            print(f"Prescription: {medical_record[2]}")
            print(f"Follow-up Date: {medical_record[3]}")
        else:
            print("No medical record found for the selected appointment.")
    else:
        print("You have no completed appointments to view medical records.")

def view_all_appointments_for_doctor(cursor, doctor_id):
    cursor.execute("""
        SELECT a.appointment_id, p.first_name, p.last_name, a.appointment_date, a.appointment_time, a.status
        FROM Appointment a
        JOIN Patient p ON a.patient_id = p.patient_id
        WHERE a.doctor_id = %s;
    """, (doctor_id,))

    appointments = cursor.fetchall()

    if not appointments:
        print("No appointments found.")
        return

    print("Your Appointments:")
    for appt in appointments:
        print(f"ID: {appt[0]}, Patient: {appt[1]} {appt[2]}, Date: {appt[3]}, Time: {appt[4]}, Status: {appt[5]}")


def view_patient_details(cursor):
    search_option = input("Do you want to search by ID or Name? (Enter 'ID' or 'Name'): ").strip().lower()

    if search_option == 'id':
        patient_id = input("Enter patient ID: ")
        cursor.execute("""
            SELECT * FROM Patient WHERE patient_id = %s;
        """, (patient_id,))
    elif search_option == 'name':
        patient_name = input("Enter patient name: ")
        cursor.execute("""
            SELECT * FROM Patient WHERE first_name ILIKE %s OR last_name ILIKE %s;
        """, (f"%{patient_name}%", f"%{patient_name}%"))
    else:
        print("Invalid option. Please enter 'ID' or 'Name'.")
        return

    patients = cursor.fetchall()

    if not patients:
        print("No patient found.")
        return

    print("Patient Details:")
    for patient in patients:
        print(f"ID: {patient[0]}, Name: {patient[1]} {patient[2]}, Gender: {patient[3]}, Age: {patient[4]}, Phone: {patient[5]}, Email: {patient[6]}, Address: {patient[7]}, Medical History: {patient[8]}")

def provide_medical_report(cursor, doctor_id):
    appointment_id = input("Enter the appointment ID: ")
    
    cursor.execute("""
        SELECT * FROM Appointment WHERE appointment_id = %s AND doctor_id = %s;
    """, (appointment_id, doctor_id))
    
    appointment = cursor.fetchone()

    if not appointment:
        print("No appointment found or does not belong to you.")
        return

    doctor_review = input("Enter doctor's review: ")
    diagnosis = input("Enter diagnosis: ")
    prescription = input("Enter prescribed medications or treatment: ")
    follow_up_date = input("Enter suggested follow-up date (YYYY-MM-DD), or leave blank: ")

    cursor.execute("""
        INSERT INTO Medical_Records (appointment_id, doctor_review, diagnosis, prescription, follow_up_date)
        VALUES (%s, %s, %s, %s, %s);
    """, (appointment_id, doctor_review, diagnosis, prescription, follow_up_date or None))

    cursor.execute("""
        UPDATE Appointment 
        SET status = 'Completed', medical_report = 'Given' 
        WHERE appointment_id = %s;
    """, (appointment_id,))


    print("Medical report provided successfully.")

def check_appointments_by_date(cursor, doctor_id):
    appointment_date = input("Enter the appointment date (YYYY-MM-DD): ")

    cursor.execute("""
        SELECT a.appointment_id, p.first_name, p.last_name, a.appointment_time, a.status
        FROM Appointment a
        JOIN Patient p ON a.patient_id = p.patient_id
        WHERE a.doctor_id = %s AND a.appointment_date = %s;
    """, (doctor_id, appointment_date))

    appointments = cursor.fetchall()

    if not appointments:
        print(f"No appointments found for {appointment_date}.")
        return

    print(f"Appointments on {appointment_date}:")
    for appt in appointments:
        print(f"ID: {appt[0]}, Patient: {appt[1]} {appt[2]}, Time: {appt[3]}, Status: {appt[4]}")
        
def view_medical_record_for_doctor(cursor, doctor_id):
    appointment_id = input("Enter the Appointment ID to view the medical record: ").strip()

    cursor.execute("""
        SELECT a.appointment_id, p.first_name, p.last_name, a.appointment_date, mr.diagnosis, mr.doctor_review, mr.prescription, mr.follow_up_date
        FROM Appointment a
        JOIN Patient p ON a.patient_id = p.patient_id
        JOIN Medical_Records mr ON a.appointment_id = mr.appointment_id
        WHERE a.doctor_id = %s AND a.appointment_id = %s
    """, (doctor_id, appointment_id))
    
    record = cursor.fetchone()
    
    if record:
        print(f"\nMedical Record for Appointment ID: {record[0]}")
        print(f"Patient Name: {record[1]} {record[2]}")
        print(f"Appointment Date: {record[3]}")
        print(f"Diagnosis: {record[4]}")
        print(f"Doctor's Review: {record[5]}")
        print(f"Prescription: {record[6]}")
        print(f"Follow-up Date: {record[7]}")
    else:
        print("No medical record found for the given appointment ID or you're not authorized to view this record.")


        
def check_manager_exists(cursor):
    cursor.execute("SELECT * FROM Manager;")
    return cursor.fetchone() is not None

def create_manager(cursor):
    print("No manager found. Let's create a new manager account.")
    first_name = input("Enter manager's first name: ")
    last_name = input("Enter manager's last name: ")
    phone_number = input("Enter manager's phone number: ")

    # Email validation
    while True:
        email = input("Enter manager's email (format: example@gmail.com): ")
        if validate_email(email):
            break
        else:
            print("Invalid email format. Please try again.")

    # Password input with dots
    password = getpass.getpass("Enter manager's password: ")

    cursor.execute("""
        INSERT INTO Manager (first_name, last_name, phone_number, email, password)
        VALUES (%s, %s, %s, %s, %s);
    """, (first_name, last_name, phone_number, email, password))

    print("Manager account created successfully!")
    return email, password 
def view_appointments_for_manager(cursor):
    cursor.execute("""
        SELECT a.appointment_id, p.first_name, p.last_name, d.first_name, d.last_name, a.appointment_date, a.appointment_time, a.status 
        FROM Appointment a
        JOIN Patient p ON a.patient_id = p.patient_id
        JOIN Doctor d ON a.doctor_id = d.doctor_id;
    """)

    appointments = cursor.fetchall()

    if not appointments:
        print("No appointments found.")
        return

    print("All Appointments:")
    for appt in appointments:
        print(f"ID: {appt[0]}, Patient: {appt[1]} {appt[2]}, Doctor: Dr. {appt[3]} {appt[4]}, Date: {appt[5]}, Time: {appt[6]}, Status: {appt[7]}")

def view_all_doctors(cursor):
    cursor.execute("""
        SELECT doctor_id, first_name, last_name, specialization, experience, appointment_price FROM Doctor;
    """)

    doctors = cursor.fetchall()

    if not doctors:
        print("No doctors found.")
        return

    print("All Doctors:")
    for doctor in doctors:
        print(f"ID: {doctor[0]}, Name: Dr. {doctor[1]} {doctor[2]}, Specialization: {doctor[3]}, Experience: {doctor[4]} years, Price: {doctor[5]}")

def view_all_patients(cursor):
    cursor.execute("""
        SELECT patient_id, first_name, last_name, gender, age, phone_number, email FROM Patient;
    """)

    patients = cursor.fetchall()

    if not patients:
        print("No patients found.")
        return

    print("All Patients:")
    for patient in patients:
        print(f"ID: {patient[0]}, Name: {patient[1]} {patient[2]}, Gender: {patient[3]}, Age: {patient[4]}, Phone: {patient[5]}, Email: {patient[6]}")

def view_all_staff(cursor):
    cursor.execute("""
        SELECT staff_id, name, role, contact_info FROM Staff;
    """)

    staff = cursor.fetchall()

    if not staff:
        print("No staff found.")
        return

    print("All Staff:")
    for member in staff:
        print(f"ID: {member[0]}, Name: {member[1]}, Role: {member[2]}, Contact: {member[3]}")

def remove_doctor(cursor,conn):
    doctor_id = input("Enter the doctor ID you want to remove: ")

    cursor.execute("""
        SELECT a.appointment_id, p.first_name, p.last_name, a.appointment_date
        FROM Appointment a
        JOIN Patient p ON a.patient_id = p.patient_id
        WHERE a.doctor_id = %s AND a.status = 'Scheduled';
    """, (doctor_id,))

    appointments = cursor.fetchall()

    if appointments:
        print("This doctor has ongoing appointments. The following patients need to be rescheduled:")
        for appt in appointments:
            print(f"Appointment ID: {appt[0]}, Patient: {appt[1]} {appt[2]}, Date: {appt[3]}")
        
        reschedule_appointments(cursor,conn, doctor_id)

        conn.commit()

    cursor.execute("""
        DELETE FROM Doctor
        WHERE doctor_id = %s;
    """, (doctor_id,))

    if cursor.rowcount > 0:
        print("Doctor removed successfully.")
    else:
        print("Doctor ID not found.")

    conn.commit()

def reschedule_appointments(cursor, conn,doctor_id):

    cursor.execute("""
        SELECT a.appointment_id, p.first_name, p.last_name, a.appointment_date, a.appointment_time
        FROM Appointment a
        JOIN Patient p ON a.patient_id = p.patient_id
        WHERE a.doctor_id = %s AND a.status = 'Scheduled';
    """, (doctor_id,))

    appointments = cursor.fetchall()

    if not appointments:
        print("No appointments to reschedule.")
        return

    print("Rescheduling the following appointments:")
    for appt in appointments:
        print(f"Appointment ID: {appt[0]}, Patient: {appt[1]} {appt[2]}, Date: {appt[3]}, Time: {appt[4]}")

        new_doctor_id = input(f"Enter new doctor ID for Appointment ID {appt[0]}: ")

        cursor.execute("""
            UPDATE Appointment
            SET doctor_id = %s
            WHERE appointment_id = %s;
        """, (new_doctor_id, appt[0]))

    print("All appointments rescheduled successfully.")

def remove_staff(cursor):
    staff_id = input("Enter the staff ID you want to remove: ")

    cursor.execute("""
        DELETE FROM Staff
        WHERE staff_id = %s;
    """, (staff_id,))

    if cursor.rowcount > 0:
        print("Staff removed successfully.")
    else:
        print("Staff ID not found.")

def add_doctor(cursor):
    first_name = input("Enter the doctor's first name: ")
    last_name = input("Enter the doctor's last name: ")
    specialization = input("Enter the doctor's specialization: ")
    phone_number=input("Enter the doctor's phone number :")
    specialty=input("Enter the doctor's specialty: ")
    experience = int(input("Enter the doctor's experience in years: "))
    appointment_price = float(input("Enter the doctor's appointment price: "))
    email = input("Enter the doctor's email: ")
    password = input("Enter the doctor's password: ")
    address=input("Enter the doctor's address: ")
    

    cursor.execute("""
        INSERT INTO Doctor (first_name, last_name, specialization, experience, appointment_price, email, password,phone_number,specialty,address)
        VALUES (%s, %s, %s, %s, %s, %s, %s,%s,%s,%s);
    """, (first_name, last_name, specialization, experience, appointment_price, email, password,phone_number,specialty,address))

    print("Doctor added successfully.")

def add_staff(cursor):
    name = input("Enter the staff member's name: ")
    role = input("Enter the staff member's role: ")
    contact_info = input("Enter the staff member's contact info: ")

    cursor.execute("""
        INSERT INTO Staff (name, role, contact_info)
        VALUES (%s, %s, %s);
    """, (name, role, contact_info))

    print("Staff member added successfully.")

def view_medical_record_for_manager(cursor):
    appointment_id = input("Enter the Appointment ID to view the medical record: ").strip()

    cursor.execute("""
        SELECT a.appointment_id, d.first_name AS doctor_first, d.last_name AS doctor_last, p.first_name AS patient_first, p.last_name AS patient_last, a.appointment_date, mr.diagnosis, mr.doctor_review, mr.prescription, mr.follow_up_date
        FROM Appointment a
        JOIN Doctor d ON a.doctor_id = d.doctor_id
        JOIN Patient p ON a.patient_id = p.patient_id
        JOIN Medical_Records mr ON a.appointment_id = mr.appointment_id
        WHERE a.appointment_id = %s
    """, (appointment_id,))
    
    record = cursor.fetchone()
    
    if record:
        print(f"\nMedical Record for Appointment ID: {record[0]}")
        print(f"Doctor: Dr. {record[1]} {record[2]}")
        print(f"Patient: {record[3]} {record[4]}")
        print(f"Appointment Date: {record[5]}")
        print(f"Diagnosis: {record[6]}")
        print(f"Doctor's Review: {record[7]}")
        print(f"Prescription: {record[8]}")
        print(f"Follow-up Date: {record[9]}")
    else:
        print("No medical record found for the given appointment ID.")
        
def add_receptionist(cursor):
    first_name = input("Enter the first name: ")
    last_name = input("Enter the last name: ")
    while True:
        email = input("Enter your email (format: example@gmail.com): ")
        if validate_email(email):
            break
        else:
            print("Invalid email format. Please try again.")
    password = getpass.getpass("Enter the password: ")
    phone_number = input("Enter the phone number: ")
    shift_time=input("Enter the shift_time (morning or night): ")
    cursor.execute("""
        INSERT INTO Receptionist (first_name, last_name, email, password, phone_number,shift_time)
        VALUES (%s, %s, %s, %s, %s,%s);
    """, (first_name, last_name, email, password, phone_number,shift_time))
    
    print(f"Receptionist {first_name} {last_name} added successfully.")

def remove_receptionist(cursor, conn):
    receptionist_id = int(input("Enter the Receptionist ID to remove: "))
    
    cursor.execute("""
        DELETE FROM Receptionist WHERE receptionist_id = %s;
    """, (receptionist_id,))
    
    conn.commit()
    print(f"Receptionist ID {receptionist_id} removed successfully.")

def view_all_receptionists(cursor):
    cursor.execute("SELECT receptionist_id, first_name, last_name, email, phone_number , shift_time FROM Receptionist;")
    receptionists = cursor.fetchall()

    if receptionists:
        print("\nList of Receptionists:")
        for receptionist in receptionists:
            print(f"ID: {receptionist[0]}, Name: {receptionist[1]} {receptionist[2]}, Email: {receptionist[3]}, Phone: {receptionist[4]}")
    else:
        print("No receptionists found.")
        
def receptionist_menu(cursor, conn, receptionist_id):
    while True:
        print("\n1. Generate Bill")
        print("2. View Bill by Appointment ID")
        print("3. Update Payment Status")
        print("4. Check Payment Status by Appointment ID")
        print("5. View All Appointments")
        print("6. View All Unpaid Bills")  
        print("7. View All Paid Bills")    
        print("8. Logout")
        
        action = input("Select an option: ").strip()
        
        if action == '1':
            appointment_id = int(input("Enter the Appointment ID: "))
            total_amount = float(input("Enter the total amount: "))
            payment_method = input("Enter the payment method (Cash/Card/Online): ")
            generate_bill(cursor, appointment_id, total_amount, payment_method, receptionist_id)
            conn.commit()
        elif action == '2':
            appointment_id = int(input("Enter the Appointment ID: "))
            view_bill_by_appointment(cursor, appointment_id)
        elif action == '3':
            bill_id = int(input("Enter the Bill ID: "))
            payment_status = input("Enter the new payment status (Paid/Unpaid/Pending): ")
            payment_date = input("Enter the payment date (YYYY-MM-DD): ")
            update_payment_status(cursor, bill_id, payment_status, payment_date)
            conn.commit()
        elif action == '4':
            appointment_id = int(input("Enter the Appointment ID: "))
            check_payment_status_by_appointment(cursor, appointment_id)
        elif action == '5':
            view_all_appointments(cursor)
        elif action == '6':
            view_unpaid_bills(cursor)  
        elif action == '7':
            view_paid_bills(cursor)  
        elif action == '8':
            print("Logged out.")
            break

def view_all_appointments(cursor):
    cursor.execute("SELECT appointment_id, patient_id, doctor_id, appointment_date, appointment_time, status FROM Appointment")
    appointments = cursor.fetchall()

    if appointments:
        print("\nList of Appointments:")
        for appointment in appointments:
            print(f"ID: {appointment[0]}, Patient ID: {appointment[1]}, Doctor ID: {appointment[2]}, Date: {appointment[3]}, Time: {appointment[4]}, Status: {appointment[5]}")
    else:
        print("No appointments found.")

            
def receptionist_login(cursor):
    while True:
        email = input("Enter your email (format: example@gmail.com): ")
        if validate_email(email):
            break
        else:
            print("Invalid email format. Please try again.")
    password = getpass.getpass("Enter your password: ")
    cursor.execute("""
        SELECT receptionist_id, first_name, last_name
        FROM Receptionist
        WHERE email = %s AND password = %s
    """, (email, password))

    receptionist = cursor.fetchone()
    
    if receptionist:
        print(f"Welcome, {receptionist[1]} {receptionist[2]}")
        return receptionist
    else:
        print("Invalid email or password.")
        return None

            
def generate_bill(cursor, appointment_id, total_amount, payment_method, receptionist_id):
    cursor.execute("""
        INSERT INTO Bills (appointment_id, total_amount, payment_method, receptionist_id)
        VALUES (%s, %s, %s, %s)
        RETURNING bill_id;
    """, (appointment_id, total_amount, payment_method, receptionist_id))
    bill_id = cursor.fetchone()[0]
    print(f"Bill generated with ID: {bill_id}")
    
def view_bill_by_appointment(cursor, appointment_id):
    cursor.execute("""
        SELECT bill_id, total_amount, payment_status, payment_date, payment_method, receptionist_id
        FROM Bills
        WHERE appointment_id = %s
    """, (appointment_id,))
    
    bill = cursor.fetchone()
    if bill:
        print(f"Bill ID: {bill[0]}")
        print(f"Total Amount: {bill[1]}")
        print(f"Payment Status: {bill[2]}")
        print(f"Payment Date: {bill[3]}")
        print(f"Payment Method: {bill[4]}")
        print(f"Processed by Receptionist ID: {bill[5]}")
    else:
        print("No bill found for the given appointment ID.")
        
def check_payment_status_by_appointment(cursor, appointment_id):
    cursor.execute("""
        SELECT b.bill_id, b.total_amount, b.payment_method, b.payment_status
        FROM Bills b  -- Ensure this matches the table name
        JOIN Appointment a ON b.appointment_id = a.appointment_id
        WHERE a.appointment_id = %s
    """, (appointment_id,))
    bill = cursor.fetchone()

    if bill:
        print(f"Bill ID: {bill[0]}, Total Amount: {bill[1]}, Payment Method: {bill[2]}, Payment Status: {bill[3]}")
    else:
        print("No bill found for this appointment.")

def view_unpaid_bills(cursor):
    cursor.execute("""
        SELECT COUNT(*)
        FROM Bills
        WHERE LOWER(payment_status) = 'unpaid' OR LOWER(payment_status) = 'pending'
    """)
    count = cursor.fetchone()[0]

    if count > 0:
        cursor.execute("""
            SELECT bill_id, total_amount, payment_method, payment_status
            FROM Bills
            WHERE LOWER(payment_status) = 'unpaid' OR LOWER(payment_status) = 'pending'
        """)
        unpaid_bills = cursor.fetchall()

        print("\nUnpaid or Pending Bills:")
        for bill in unpaid_bills:
            print(f"Bill ID: {bill[0]}, Total Amount: {bill[1]}, Payment Method: {bill[2]}, Payment Status: {bill[3]}")
    else:
        print("No unpaid or pending bills found.")



def view_paid_bills(cursor):
    cursor.execute("""
        SELECT COUNT(*)
        FROM Bills
        WHERE LOWER(payment_status) = 'paid'
    """)
    count = cursor.fetchone()[0]

    if count > 0:
        cursor.execute("""
            SELECT bill_id, total_amount, payment_method, payment_status
            FROM Bills
            WHERE LOWER(payment_status) = 'paid'
        """)
        paid_bills = cursor.fetchall()

        print("\nPaid Bills:")
        for bill in paid_bills:
            print(f"Bill ID: {bill[0]}, Total Amount: {bill[1]}, Payment Method: {bill[2]}, Payment Status: {bill[3]}")
    else:
        print("No paid bills found.")



def update_payment_status(cursor, bill_id, payment_status, payment_date):
    cursor.execute("""
        UPDATE Bills
        SET payment_status = %s, payment_date = %s
        WHERE bill_id = %s
    """, (payment_status, payment_date, bill_id))
    print(f"Payment status updated for Bill ID: {bill_id}")

    
# Main function to run the program
def main():
    conn = connect()
    cursor = conn.cursor()

    while True:
        user_type = input("Are you a Patient, Doctor, Manager, or Receptionist? (Enter 'Patient', 'Doctor', 'Manager', 'Receptionist', or 'Exit' to quit): ").strip().lower()
        
        if user_type == 'exit':
            break

        if user_type == 'patient':
            action = input("Do you want to Register or Log in? (Enter 'Register' or 'Log in'): ").strip().lower()

            if action == 'register':
                register_patient(cursor)
                conn.commit()
            elif action == 'log in':
                email = input("Enter your email: ")
                password = getpass.getpass("Enter your password: ")
                patient = authenticate_user(cursor, 'patient', email, password)

                if patient:
                    print(f"Welcome, {patient[1]} {patient[2]}")
                    while True:
                        print("\n1. Book Appointment")
                        print("2. View Appointments")
                        print("3. Change Appointment")
                        print("4. Cancel Appointment")
                        print("5. View Medical Report")
                        print("6. Logout")

                        action = input("Select an option: ").strip()
                        if action == '1':
                            book_appointment(cursor, patient[0])
                            conn.commit()
                        elif action == '2':
                            view_appointments(cursor, patient[0])
                        elif action == '3':
                            change_appointment(cursor, patient[0])
                            conn.commit()
                        elif action == '4':
                            cancel_appointment(cursor, patient[0])
                            conn.commit()
                        elif action == '5':
                            check_medical_record(cursor, patient[0])
                            conn.commit()
                        elif action == '6':
                            print("Logged out.")
                            break
                else:
                    print("Invalid email or password.")
                    
        elif user_type == 'doctor':
            email = input("Enter your email: ")
            password = getpass.getpass("Enter your password: ")
            doctor = authenticate_user(cursor, 'doctor', email, password)

            if doctor:
                print(f"Welcome, Dr. {doctor[1]} {doctor[2]}")
                while True:
                    print("\n1. View All Appointments")
                    print("2. View Patient Details by ID or Name")
                    print("3. Provide Medical Report")
                    print("4. Check Appointments by Date")
                    print("5. View Medical Records")
                    print("6. Logout")

                    action = input("Select an option: ").strip()
                    if action == '1':
                        view_all_appointments_for_doctor(cursor, doctor[0])
                    elif action == '2':
                        view_patient_details(cursor)
                    elif action == '3':
                        provide_medical_report(cursor, doctor[0])
                        conn.commit()
                    elif action == '4':
                        check_appointments_by_date(cursor, doctor[0])
                    elif action == '5':
                        view_medical_record_for_doctor(cursor, doctor[0])
                    elif action == '6':
                        print("Logged out.")
                        break
            else:
                print("Invalid email or password.")
                
        elif user_type == 'manager':
            if check_manager_exists(cursor):
                email = input("Enter your email: ")
                password = getpass.getpass("Enter your password: ")
                manager = authenticate_user(cursor, 'manager', email, password)

                if manager:
                    print(f"Welcome, {manager[1]} {manager[2]}")
                    while True:
                        print("\n1. View All Appointments")
                        print("2. View All Doctors")
                        print("3. View All Patients")
                        print("4. View All Staff")
                        print("5. Add Doctor")
                        print("6. Add Staff")
                        print("7. Remove Doctor")
                        print("8. Remove Staff")
                        print("9. View Medical Records")
                        print("10. View All Receptionists")  
                        print("11. Add Receptionist")        
                        print("12. Remove Receptionist")      
                        print("13. Logout")

                        action = input("Select an option: ").strip()
                        if action == '1':
                            view_appointments_for_manager(cursor)
                        elif action == '2':
                            view_all_doctors(cursor)
                        elif action == '3':
                            view_all_patients(cursor)
                        elif action == '4':
                            view_all_staff(cursor)
                        elif action == '5':
                            add_doctor(cursor)
                            conn.commit()
                        elif action == '6':
                            add_staff(cursor)
                            conn.commit()
                        elif action == '7':
                            remove_doctor(cursor, conn)
                        elif action == '8':
                            remove_staff(cursor)
                            conn.commit()
                        elif action == '9':
                            view_medical_record_for_manager(cursor)
                        elif action == '10':
                            view_all_receptionists(cursor)  
                        elif action == '11':
                            add_receptionist(cursor)  
                            conn.commit()
                        elif action == '12':
                            remove_receptionist(cursor, conn)  
                            conn.commit()
                        elif action == '13':
                            print("Logged out.")
                            break
            else:
                email, password = create_manager(cursor)
                conn.commit()

        elif user_type == 'receptionist':
            receptionist = receptionist_login(cursor)
            if receptionist:
                receptionist_menu(cursor,conn, receptionist[0])

if __name__ == "__main__":
    main()


Are you a Patient, Doctor, Manager, or Receptionist? (Enter 'Patient', 'Doctor', 'Manager', 'Receptionist', or 'Exit' to quit): Manager
Enter your email: har@gmail.com
Enter your password: ········
Are you a Patient, Doctor, Manager, or Receptionist? (Enter 'Patient', 'Doctor', 'Manager', 'Receptionist', or 'Exit' to quit): Exit
